In [1]:
# Parameters
name = "asd"
surname = "asd"
sector = "S\u0259hhiyy\u0259"
field = "H\u0259kiml\u0259rin say\u0131"
start_year = 2026
end_year = 2028

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
import matplotlib
matplotlib.use('Agg')
warnings.filterwarnings('ignore')

In [3]:
# Prophet model (alternative implementation)
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    PROPHET_AVAILABLE = False
    print("Prophet not available, using alternative forecasting methods")

In [4]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as patches
from datetime import datetime, timedelta

In [5]:
# Set matplotlib to use available fonts
plt.rcParams['font.family'] = ['DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)
# Enable Unicode support
plt.rcParams['axes.unicode_minus'] = False

In [6]:
# Read the CSV data
def read_healthcare_data():
    # Read CSV data
    df = pd.read_csv('output_cleaned.csv')
    df['Tarix'] = pd.to_datetime(df['Tarix'], format='%Y')
    df.set_index('Tarix', inplace=True)
    return df

In [7]:
def linear_trend_forecast(series, periods=3):
    """Linear trend forecasting as fallback"""
    X = np.arange(len(series)).reshape(-1, 1)
    y = series.values

    model = LinearRegression()
    model.fit(X, y)

    future_X = np.arange(len(series), len(series) + periods).reshape(-1, 1)
    forecast = model.predict(future_X)

    # Simple confidence interval estimation
    residuals = y - model.predict(X)
    std_error = np.std(residuals)
    conf_int = np.column_stack([forecast - 1.96*std_error, forecast + 1.96*std_error])

    return forecast, conf_int, model

In [8]:
# Time series forecasting functions
def arima_forecast(series, periods=3, order=(1,1,1)):
    """ARIMA forecasting"""
    try:
        model = ARIMA(series, order=order)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=periods)
        conf_int = fitted_model.get_forecast(steps=periods).conf_int()
        return forecast.values, conf_int.values, fitted_model
    except:
        try:
            # Fallback to simple ARIMA
            model = ARIMA(series, order=(1,1,0))
            fitted_model = model.fit()
            forecast = fitted_model.forecast(steps=periods)
            conf_int = fitted_model.get_forecast(steps=periods).conf_int()
            return forecast.values, conf_int.values, fitted_model
        except:
            # Final fallback to linear trend
            return linear_trend_forecast(series, periods)

In [9]:
def prophet_forecast(series, periods=3):
    """Prophet forecasting (if available)"""
    if not PROPHET_AVAILABLE:
        return linear_trend_forecast(series, periods)

    try:
        df = pd.DataFrame({
            'ds': series.index,
            'y': series.values
        })

        model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
        model.fit(df)

        future = model.make_future_dataframe(periods=periods, freq='Y')
        forecast = model.predict(future)

        forecast_values = forecast.tail(periods)['yhat'].values
        conf_int = forecast.tail(periods)[['yhat_lower', 'yhat_upper']].values

        return forecast_values, conf_int, model
    except:
        return linear_trend_forecast(series, periods)

In [10]:
def random_forest_forecast(series, periods=3):
    """Random Forest forecasting"""
    # Create features (lagged values)
    n_lags = min(5, len(series) // 2)
    X, y = [], []

    for i in range(n_lags, len(series)):
        X.append(series.values[i-n_lags:i])
        y.append(series.values[i])

    X, y = np.array(X), np.array(y)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)

    # Generate forecasts
    forecasts = []
    last_values = series.values[-n_lags:]

    for _ in range(periods):
        pred = model.predict([last_values])[0]
        forecasts.append(pred)
        last_values = np.append(last_values[1:], pred)

    return np.array(forecasts), None, model

In [11]:
# Analysis functions
def perform_analysis():
    df = read_healthcare_data()
    # Key specialties to analyze
    specialties = [
        'Həkimlərin sayı - cəmi',
        'terapevtlər',
        'cərrahlar',
        'pediatrlar',
        'stomatoloq və diş həkimləri'
    ]
    results = {}

    for specialty in specialties:
        series = df[specialty]

        # ARIMA forecast
        arima_pred, arima_conf, arima_model = arima_forecast(series)

        # Prophet forecast
        prophet_pred, prophet_conf, prophet_model = prophet_forecast(series)

        # Random Forest forecast
        rf_pred, rf_conf, rf_model = random_forest_forecast(series)

        # Linear trend forecast
        linear_pred, linear_conf, linear_model = linear_trend_forecast(series)

        results[specialty] = {
            'historical': series,
            'arima': {'forecast': arima_pred, 'conf_int': arima_conf, 'model': arima_model},
            'prophet': {'forecast': prophet_pred, 'conf_int': prophet_conf, 'model': prophet_model},
            'rf': {'forecast': rf_pred, 'conf_int': rf_conf, 'model': rf_model},
            'linear': {'forecast': linear_pred, 'conf_int': linear_conf, 'model': linear_model}
        }

    return results

In [29]:
def create_forecast_plot(specialty, data, ax):
    """Create forecast visualization for a specialty with custom methods for each"""
    historical = data['historical']

    # Plot historical data
    ax.plot(historical.index, historical.values, 'o-', label='Tarixi məlumatlar',
            color='blue', linewidth=2, markersize=4)

    # Future years
    future_years = pd.date_range(start='2025', periods=3, freq='Y')

    # Define which methods to keep for each specialty
    methods_config = {
        'Həkimlərin sayı - cəmi': ['arima', 'linear'],
        'terapevtlər': ['arima', 'rf', 'prophet'],
        'cərrahlar': ['arima', 'rf', 'linear'],
        'pediatrlar': ['arima', 'linear'],
        'stomatoloq və diş həkimləri': ['arima', 'prophet']
    }

    # Method display names and colors
    method_info = {
        'arima': {'name': 'ARIMA', 'color': 'red'},
        'prophet': {'name': 'Prophet', 'color': 'green'},
        'rf': {'name': 'Random Forest', 'color': 'orange'},
        'linear': {'name': 'Xətti Trend', 'color': 'purple'}
    }

    # Get methods to plot for this specialty
    methods_to_plot = methods_config.get(specialty, ['arima', 'prophet'])

    # Plot forecasts for selected methods
    for method in methods_to_plot:
        info = method_info[method]
        forecast = data[method]['forecast']

        ax.plot(future_years, forecast, 'o--',
                label=f'{info["name"]} proqnozu',
                color=info['color'], linewidth=2, markersize=6)

        # Add confidence intervals if available
        conf_int = data[method]['conf_int']
        if conf_int is not None:
            try:
                if hasattr(conf_int, 'values'):
                    conf_int = conf_int.values
                if conf_int.ndim == 2 and conf_int.shape[1] >= 2:
                    ax.fill_between(future_years, conf_int[:, 0], conf_int[:, 1],
                                  alpha=0.2, color=info['color'])
                elif conf_int.ndim == 1 and len(conf_int) >= 2:
                    margin = np.std(forecast) * 0.5
                    ax.fill_between(future_years, forecast - margin, forecast + margin,
                                  alpha=0.2, color=info['color'])
            except:
                pass  # Skip if error

    ax.set_title(f'{specialty} - Zaman Seriyası Analizi və Proqnoz',
                fontsize=14, fontweight='bold')
    ax.set_xlabel('İl', fontsize=12)
    ax.set_ylabel('Həkimlərin sayı', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

    # Add trend line
    from scipy import stats
    years_numeric = np.arange(len(historical))
    slope, intercept, r_value, p_value, std_err = stats.linregress(years_numeric, historical.values)

    # Extend trend line to future
    all_years_numeric = np.arange(len(historical) + 3)
    trend_line = slope * all_years_numeric + intercept
    all_years = list(historical.index) + list(future_years)

    # Add explanations for predictions in each visual
    visual_explanations = {
    'Həkimlərin sayı - cəmi': (
        "Terapevtlərin sayı əvvəllər az-az artdı, amma son vaxtlar bu artım dayandı və bəzən azalma müşahidə edilir. "
        "Bunun əsas səbəbi ailə həkimliyi sisteminin geniş yayılmasıdır. İndi terapevtlərin bəzi işlərini ümumi "
        "təyinatlı həkimlər də görür. Təxminən 2010-cu ildən sonra terapevtlərin sayında artım yavaşlayıb. Proqnozlara görə, "
        "2028-ci ilə qədər terapevtlərin sayı ya çox dəyişməyəcək, ya da bir qədər azalacaq. Bəzi modellər bu azalma tendensiyasını "
        "daha yumşaq göstərir. Azalma səbəbləri arasında gənclərin bu sahəyə marağının az olması, terapevtlərin yaşlanması və iş "
        "yükünün çox olması yer alır. Gələcəkdə terapevtlərin iş yükü digər həkim ixtisasları arasında bölüşdürülə bilər. Ümumilikdə, "
        "yaxın illərdə terapevtlərin sayında azalma ehtimalı var."),

    'terapevtlər': (
        "Terapevtlərin sayı illər ərzində bir az artsa da, son zamanlar bu artım dayanıb və azalma hiss olunur. "
        "Bu vəziyyət əsasən ailə həkimliyi sisteminin yayılması ilə bağlıdır. Əvvəllər terapevtlərin gördüyü bir çox işi "
        "indi ümumi təyinatlı həkimlər də yerinə yetirir. Təxminən 2010-cu ildən sonra artım yavaşlayıb. "
        "Hazırda bir çox model bu azalma tendensiyasının davam edəcəyini göstərir. Məsələn, ARIMA və Prophet modelləri göstərir ki, "
        "2028-ci ilə qədər terapevtlərin sayı az qala dəyişməyəcək və ya bir qədər də azalacaq. Random Forest modeli isə bunu bir az fərqli qiymətləndirir, "
        "əvvəlki illərdəki dəyişiklikləri də nəzərə alaraq nisbətən yumşaq nəticələr verir. "
        "Bu azalma həm də gənclərin bu sahəyə marağının azalması, terapevtlərin yaşlanması və iş yükünün artması ilə bağlı ola bilər. "
        "Gələcəkdə terapevtlərin işi digər həkim ixtisasları arasında bölüşdürülə bilər. "
        "Bütün bunları nəzərə alsaq, proqnozlar göstərir ki, yaxın illərdə terapevtlərin sayında azalma ehtimalı qalmaqdadır."),

    'cərrahlar': (
        "Cərrahların sayı uzun illər ərzində sabit qalıb və ciddi dalğalanma müşahidə olunmayıb. "
        "Bu sahədə artım tempi digər ixtisaslara nisbətən daha zəifdir. "
        "Əsas səbəb cərrah ixtisasının uzunmüddətli və çətin təhsil tələb etməsidir. "
        "Eyni zamanda cərrah olmaq üçün təcrübə toplamaq və yüksək səviyyəli bacarıqlar qazanmaq vacibdir. "
        "Son illərdə texnologiyaların, xüsusilə də minimal invaziv və robotik üsulların inkişafı cərrahların işini dəyişib. "
        "Bu texnologiyalar həm imkanları genişləndirib, həm də yeni təlimlərə ehtiyac yaradıb. "
        "Yeni sistemlərə uyğunlaşmaq bəzi mütəxəssislər üçün çətinlik yaradır. "
        "Buna görə də sahəyə maraq göstərənlərin sayı çox sürətlə artmır. "
        "ARIMA və trend modelləri göstərir ki, 2028-ci ilə qədər cərrahların sayında yavaş və stabil artım gözlənilir. "
        "Random Forest modelində bu artım bir qədər daha nikbin qiymətləndirilir. "
        "Gələcəkdə bəzi texnologiyalar bəzi əməliyyatlarda insan ehtiyacını azaltsa da, kritik və yüksək riskli əməliyyatlarda hələ də cərrahlara ehtiyac qalacaq. "
        "Təhsil sistemində texnologiyaya uyğun yeni proqramlar tətbiq olunarsa, bu sahəyə maraq arta bilər. "
        "Bölgələrdə ixtisaslı cərrahların sayı hələ də paytaxtla müqayisədə azdır. "
        "Əhalinin yaşlanması və xroniki xəstəliklərin artması da cərrah xidmətlərinə olan tələbi artıra bilər."),

    'pediatrlar': (
        "Pediatrların sayı son illərdə azalmağa başlayıb. "
        "Əvvəllər sabit bir artım tempi olsa da, son dövrlərdə bu temp dayanıqlı şəkildə aşağı düşüb. "
        "Xüsusilə 2020-ci ildən sonra bu azalma daha da nəzərə çarpır. "
        "Bunun əsas səbəblərindən biri ailə həkimlərinin uşaq xəstəliklərini də əhatə etməsidir. "
        "Belə olduqda, valideynlər birbaşa pediatra yox, ümumi təyinatlı həkimlərə müraciət edirlər. "
        "Bundan əlavə, gənclərin bu ixtisasa marağı azalıb. "
        "Pediatr ixtisası maddi baxımdan daha az gəlirli olduğu üçün seçilmə faizi də aşağıdır. "
        "Hazırda çalışan pediatrların yaş ortalaması artıb və bu da yaxın illərdə pensiyaya çıxma hallarının çoxalmasına gətirib çıxara bilər. "
        "ARIMA və xətti trend modelləri 2028-ci ilə qədər azalma tendensiyasının davam edəcəyini göstərir. "
        "Bəzi bölgələrdə pediatr çatışmazlığı artıq müşahidə olunur. "
        "Bu, uşaq sağlamlığı baxımından ciddi təhlükə yarada bilər. "
        "Əgər bu sahəyə strateji yanaşma göstərilməzsə, gələcəkdə peşəkar kadr çatışmazlığı daha da kəskinləşə bilər. "
        "Təklif olunan həllər arasında rezidentura yerlərinin artırılması və bölgələrdə çalışanlara əlavə stimul verilməsi ola bilər. "
        "Əhali artdıqca və uşaqların sayı çoxaldıqca, bu sahəyə olan ehtiyac da sabit qalır və hətta artır. "
        "Buna görə də pediatrların sayını qoruyub saxlamaq və artırmaq vacib məsələdir."),

    'stomatoloq və diş həkimləri': (
        "Son 15 ildə stomatoloq və diş həkimlərinin sayı nəzərəçarpacaq dərəcədə artıb. "
        "Bu artımın əsas səbəblərindən biri özəl sektorun inkişafı və estetik diş xidmətlərinə olan marağın çoxalmasıdır. "
        "Əhali arasında estetik görünüşə və ağız sağlamlığına diqqət artdıqca bu sahəyə tələb də artıb. "
        "Əhalinin gəlir səviyyəsinin yüksəlməsi və profilaktik diş baxımına olan maraq bu artımı gücləndirib. "
        "Yeni texnologiyaların tətbiqi, məsələn, lazerlə müalicə və 3D çap texnikası da stomatoloji sahəyə dinamika gətirib. "
        "ARIMA və Prophet modelləri göstərir ki, bu artım 2028-ci ilə qədər də davam edəcək. "
        "Trend analizləri də eyni istiqamətdə proqnoz verir. "
        "Stomatologiya həm gəlirli, həm də geniş bazara sahib sahə olduğu üçün yeni məzunların çoxu bu ixtisası seçir. "
        "Bu sahədə rəqabət güclüdür, bu da xidmət keyfiyyətini və ixtisaslaşmanı artırır. "
        "Eyni zamanda keyfiyyətli təhsilin verilməsi və etik qaydaların qorunması da vacib şərtlərdən biridir. "
        "Əgər təhsil sistemi stomatoloqların hazırlığını yüksək səviyyədə təmin edərsə, bu artım dayanıqlı olacaq. "
        "Bölgələrdə də bu xidmətlərin genişlənməsi müşahidə olunur. "
        "Diş sağlamlığına dair maarifləndirmə kampaniyaları da bu tendensiyanı dəstəkləyir. "
        "Yekun olaraq, stomatoloq və diş həkimlərinin sayının artımı davam edir və yaxın illərdə də bu artımın davam edəcəyi gözlənilir.")}

    # Format these texts as paragraphs
    import textwrap
    explanations = visual_explanations.get(specialty, "")
    wrapped_texts = textwrap.fill(explanations, width=100)

    ax.plot(all_years, trend_line, '--', color='gray', alpha=0.7,
            label=f'Trend (R²={r_value**2:.3f})')
    ax.legend(fontsize=9)

    # Add explanation text below the plot
    ax.text(0.5, -0.25, wrapped_texts,
            ha='center', va='top', transform=ax.transAxes,
            fontsize=10, style='italic', color='#555555')
    # Add values for each corresponding years
    for year, value in zip(historical.index, historical.values):
      ax.annotate(f'{(value/10)}K', xy=(year, value), xytext=(0, 3),
                  fontsize=2, ha='right', textcoords='offset points',
                  color='black', weight='bold')

In [30]:
def create_summary_statistics(results):
    """Create summary statistics table"""
    summary_data = []

    for specialty, data in results.items():
        historical = data['historical']

        # Calculate statistics
        mean_val = historical.mean()
        std_val = historical.std()
        trend = (historical.iloc[-1] - historical.iloc[0]) / len(historical)

        # Average forecast across methods
        forecasts = []
        for method in ['arima', 'prophet', 'rf', 'linear']:
            try:
                method_forecast = data[method]['forecast']
                if hasattr(method_forecast, '__len__') and len(method_forecast) >= 3:
                    forecasts.append(method_forecast)
            except:
                continue

        if forecasts:
            avg_forecast_2025 = np.mean([f[0] for f in forecasts])
            avg_forecast_2026 = np.mean([f[1] for f in forecasts])
            avg_forecast_2027 = np.mean([f[2] for f in forecasts])
        else:
            # Fallback values if no forecasts available
            avg_forecast_2025 = historical.iloc[-1]
            avg_forecast_2026 = historical.iloc[-1]
            avg_forecast_2027 = historical.iloc[-1]

        summary_data.append({
            'İxtisas': specialty,
            'Orta (1991-2024)': f"{mean_val:.0f}",
            'Standart sapma': f"{std_val:.0f}",
            'İllik trend': f"{trend:.1f}",
            '2025 proqnozu': f"{avg_forecast_2025:.0f}",
            '2026 proqnozu': f"{avg_forecast_2026:.0f}",
            '2027 proqnozu': f"{avg_forecast_2027:.0f}"
        })

    return pd.DataFrame(summary_data)

In [31]:
def draw_summary_figure(pdf, total_doctors, specialties_2024, pie_labels, pie_sizes, growth_rates):
    """A4 səhifəsində 4 qrafiki estetik şəkildə çəkmək və PDF-ə əlavə etmək"""
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    import numpy as np

    fig = plt.figure(figsize=(8.3, 11.7))  # A4 ölçüsü
    gs = gridspec.GridSpec(4, 1, height_ratios=[1, 1, 1, 1])
    fig.subplots_adjust(hspace=0.5)

    # Qrafik 1 – Ümumi Həkimlərin Sayı
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(total_doctors.index.year, total_doctors.values, color='#1976D2', linewidth=2, marker='o', markersize=4)
    ax1.set_title('Ümumi Həkimlərin Sayı (1991-2024)', fontsize=12, fontweight='bold', pad=8)
    ax1.set_xlabel('İl', fontsize=9)
    ax1.set_ylabel('Həkimlərin sayı', fontsize=9)
    ax1.grid(True, alpha=0.3)
    ax1.annotate(f"{int(total_doctors.values[-1])}",
                 xy=(total_doctors.index.year[-1], total_doctors.values[-1]),
                 xytext=(5, 0), textcoords='offset points',
                 fontsize=8, fontweight='bold', color='#1976D2')
    ax1.tick_params(axis='both', labelsize=8)

    # Qrafik 2 – Barh: İxtisaslara görə bölgü
    # Barh Chart — ixtisaslar üzrə bölgü
    ax2 = fig.add_subplot(gs[1])
    bars = ax2.barh(list(specialties_2024.keys()), list(specialties_2024.values()), color='#43A047', alpha=0.85)

    ax2.set_title('2024-cü İldə İxtisaslar üzrə Bölgü', fontsize=11, fontweight='bold', pad=6)
    ax2.set_xlabel('Həkimlərin sayı', fontsize=9)
    ax2.tick_params(axis='both', labelsize=8)
    ax2.set_xlim(0, max(specialties_2024.values()) * 1.2)  # sağda boşluq

    for label in ax2.get_yticklabels():
        label.set_horizontalalignment('right')  # uzun yazılar sola çıxmasın

    for bar in bars:
        ax2.text(bar.get_width() + max(specialties_2024.values()) * 0.02,
                bar.get_y() + bar.get_height()/2,
                f"{int(bar.get_width())}", va='center', fontsize=7.5, fontweight='bold')

    # Qrafik 3 – Pie chart
    ax3 = fig.add_subplot(gs[2])
    pie_colors = plt.cm.Paired(np.linspace(0, 1, len(pie_labels)))
    ax3.pie(pie_sizes, labels=pie_labels, autopct='%1.0f%%', startangle=140,
            colors=pie_colors, textprops={'fontsize': 8}, pctdistance=0.75, radius=0.65)
    ax3.set_title('2024-cü il üzrə İxtisasların Paylanması', fontsize=11, fontweight='bold', pad=6)

    # Qrafik 4 – Artım Dərəcəsi
    ax4 = fig.add_subplot(gs[3])
    growth_colors = ['#388E3C' if x > 0 else '#D32F2F' for x in growth_rates.values()]
    bars2 = ax4.bar(list(growth_rates.keys()), list(growth_rates.values()), color=growth_colors, alpha=0.85)
    ax4.set_title('1991-2024 Artım Dərəcəsi (%)', fontsize=12, fontweight='bold', pad=6)
    ax4.set_ylabel('Artım faizi', fontsize=9)
    ax4.tick_params(axis='x', labelsize=8, rotation=30)
    ax4.tick_params(axis='y', labelsize=8)
    for bar in bars2:
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"{bar.get_height():.1f}%",
                 ha='center', va='bottom', fontsize=7.5, fontweight='bold')
    ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)

    # PDF-ə əlavə et
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()

In [15]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.3 MB/s eta 0:00:00


In [32]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
import numpy as np
from PyPDF2 import PdfMerger

try:
    from app import params
except ImportError:
    params = None


def generate_pdf_report(results):
    """Generate comprehensive PDF report"""
    A4_SIZE = (8.3, 11.7)
    generated_pdf_path = f"{sector}_Sektoru_Hesabat_{end_year}.pdf"
    additional_pdf_path = "Qeydler.pdf"
    merged_pdf_path     = "Birləşdirilmiş_Hesabat.pdf"

    # First create the title page separately
    title_pdf_path = f"Title_{sector}_Sektoru_{end_year}.pdf"

    with PdfPages(title_pdf_path) as pdf:
        fig, ax = plt.subplots(figsize=A4_SIZE)
        ax.axis('off')
        # Dynamic title & author
        ax.text(0.5, 0.90, f'{sector} Sektoru Hesabat {end_year}',
                ha='center', fontsize=24, fontweight='bold')
        ax.text(0.5, 0.85,
                f'Zaman Seriyası Analizi və Maşın Öyrənməsi Proqnozları ({start_year}–{end_year})',
                ha='center', fontsize=16)
        ax.text(0.5, 0.80, f'Hazırladı: {name} {surname}',
                ha='center', fontsize=14)
        ax.text(0.5, 0.75,
                f'Tarix: {datetime.now().strftime("%d.%m.%Y")}',
                ha='center', fontsize=12)

        rect = patches.Rectangle((0.10, 0.65), 0.80, 0.05,
                                 linewidth=2, edgecolor='blue',
                                 facecolor='lightblue', alpha=0.3)
        ax.add_patch(rect)
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()

    # Create the main content pages
    content_pdf_path = f"Content_{sector}_Sektoru_{end_year}.pdf"

    with PdfPages(content_pdf_path) as pdf:
        # Summary page with statistics
        total_doctors = results['Həkimlərin sayı - cəmi']['historical']
        specialties_2024 = {}
        for spec, data in results.items():
            if spec != 'Həkimlərin sayı - cəmi':
                specialties_2024[spec] = data['historical'].iloc[-1]

        pie_labels = list(specialties_2024.keys())
        pie_sizes = list(specialties_2024.values())

        growth_rates = {}
        for spec, data in results.items():
            if spec != 'Həkimlərin sayı - cəmi':
                hist = data['historical']
                growth_rate = ((hist.iloc[-1] - hist.iloc[0]) / hist.iloc[0]) * 100
                growth_rates[spec] = growth_rate

        fig, axs = plt.subplots(2, 2, figsize=(18, 12))
        fig.suptitle('STATİSTİK MƏLUMATLAR - VİZUAL İCMAL', fontsize=28, fontweight='bold', color='black', y=0.98)

        draw_summary_figure(pdf, total_doctors, specialties_2024, pie_labels, pie_sizes, growth_rates)

        # Individual specialty forecasts
        for specialty, data in results.items():
            fig = plt.figure(figsize=A4_SIZE)
            ax = fig.add_axes([0.1, 0.5, 0.87, 0.45])
            create_forecast_plot(specialty, data, ax)
            plt.tight_layout()
            pdf.savefig(fig)
            plt.close()

    # Merge PDFs in the correct order: Title -> Notes -> Content
    merger = PdfMerger()
    merger.append(title_pdf_path)         # Title page first
    merger.append(additional_pdf_path)    # Notes second (right after title)
    merger.append(content_pdf_path)       # Content pages last
    merger.write(merged_pdf_path)
    merger.close()

    # Clean up temporary files (optional)
    import os
    try:
        os.remove(title_pdf_path)
        os.remove(content_pdf_path)
    except:
        pass  # If files don't exist or can't be deleted, continue

print("PDF hesabatı yaradılır...")
results = perform_analysis()
generate_pdf_report(results)
print("Hesabat tamamlandı: 'Birləşdirilmiş_Hesabat.pdf'")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpo7gpt9l4/dwcc3glq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpo7gpt9l4/1rmi148t.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=17319', 'data', 'file=/tmp/tmpo7gpt9l4/dwcc3glq.json', 'init=/tmp/tmpo7gpt9l4/1rmi148t.json', 'output', 'file=/tmp/tmpo7gpt9l4/prophet_modelgar0v_qk/prophet_model-20250616144445.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
14:44:45 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


PDF hesabatı yaradılır...


14:44:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpo7gpt9l4/aqzwi2mu.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpo7gpt9l4/ejguhnsz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=22084', 'data', 'file=/tmp/tmpo7gpt9l4/aqzwi2mu.json', 'init=/tmp/tmpo7gpt9l4/ejguhnsz.json', 'output', 'file=/tmp/tmpo7gpt9l4/prophet_modelsdovptdh/prophet_model-20250616144446.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
14:44:46 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:44:46 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpo7gpt9l4/c9_61e4b.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpo7gpt9l4/7k2_z4nw.json
DEBUG:cmdstanpy:id

Hesabat tamamlandı: 'Birləşdirilmiş_Hesabat.pdf'
